In [ ]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
###
# 務必先手動，開啟瀏覽器進去shutterstock，登入帳號
# 這樣才能取得 cookies，用於自動化登入
###
#Step 1: 開啟 https://submit.shutterstock.com/upload
#Step 2: 找到 CSV 上傳區塊 → 送出本機檔案路徑
#Step 3: 等待上傳完成進度條消失
#Step 4: 點擊「Apply metadata」
#Step 5: 等待套用完成
#[選配] Step 6: 勾選全部 → 點 Submit → 確認送審成功

# === 設定 ===
CSV_PATH = "/Users/donpanic/Documents/shutterstock_meta.csv"
CHROME_BIN = "/Users/donpanic/Documents/programming/chrome-mac-arm64/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
USER_DATA_DIR = "/Users/donpanic/Library/Application Support/Google/Chrome for Testing"
PROFILE_DIR = "/Users/donpanic/Documents/programming/chrome-mac-arm64/Google Chrome for Testing"  # ✅ 使用你剛

In [ ]:
# === 啟動瀏覽器 ===
options = uc.ChromeOptions()
options.binary_location = CHROME_BIN
options.add_argument(f"--user-data-dir={USER_DATA_DIR}")
options.add_argument("--profile-directory=Default")
options.add_argument("--disable-blink-features=AutomationControlled")  # Stealth 強化

driver = uc.Chrome(options=options, headless=False)
wait = WebDriverWait(driver, 30)

# === 1. 開啟 Shutterstock 上傳頁 ===
driver.get("https://submit.shutterstock.com/upload")
print("⏳ 開啟 upload 頁面中...")

In [ ]:
# === 2. 上傳 CSV 檔案 ===
try:
    csv_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file'][accept='.csv']"))
    )
    csv_input.send_keys(CSV_PATH)
    print(f"📤 已選擇 CSV 檔：{CSV_PATH}")
except Exception as e:
    print("❌ 找不到 CSV 上傳欄位，可能是未登入或頁面失敗。")
    driver.quit()
    raise e

In [ ]:
# === 3. 等待「Apply metadata」按鈕可點擊並點擊 ===
try:
    apply_btn = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Apply metadata')]"))
    )
    time.sleep(1.2)  # 模擬人類操作延遲
    apply_btn.click()
    print("✅ 點擊 Apply metadata 成功")
except Exception as e:
    print("❌ 找不到或無法點擊 Apply metadata")
    driver.quit()
    raise e

# === 4. 等待套用完成（進度條消失或成功 toast）===
try:
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".metadata-upload-progress")))
    print("🎉 Metadata 套用完成！")
except:
    print("⚠️ 未偵測到進度條消失，請手動確認結果")

# === 結束 ===
input("✅ 流程完成，你可以按 Enter 關閉瀏覽器")
driver.quit()